# EK381 Exploratory Data Analysis <br> Lab 8: Detection and Classification

### This lab was completed by Rawisara Chairat.

**Overall, the goal of this lab is to get you comfortable working with decision rules and errors. You should make sure to understand how to determine when a data point will result in an error, and how to estimate the probability of error as the fraction of misclassified points.**

**Also, when you are ready to turn in the lab, please be sure to clear the output of all cells. Gradescope has trouble with larger Jupyter notebooks, and this one will be large if all the images are preserved. There is a graded problem at the end that ony asks you to clear cell outputs, with instructions.**

This lab introduces the important application of detection, which is also referred to as classification in machine learning. The key idea is that we are trying to decide between two options (or potentially more) given an observation. A probabilistic approach to this application is to select probability models that try to explain how the observations are generated. Specifically, we select a conditional PDF $f_{Y|H_0}(y)$ that models how observation $Y$ is generated under hypothesis $H_0$ and a conditional PDF $f_{Y|H_1}(y)$ that models how observation $Y$ is generated under hypothesis $H_1$. We sometimes refer to $f_{Y|H_0}(y)$ and $f_{Y|H_1}(y)$ as **likelihoods**. In class, we learned about the likelihood ratio
$$L(y)= \frac{f_{Y|H_1}(y)}{f_{Y|H_0}(y)}$$ and the maximum likelihood (ML) rule, which chooses the hypothesis that best explains the observation,
$$D_{\text{ML}}(y) = \begin{cases}1 & L(y) \geq 1, \\ 0 & L(y) < 1. \end{cases}$$ 

For example, say we model our observations from hypothesis 0 as Gaussian(-1,1) and our observations from hypothesis 1 as Gaussian(+1,1). Then, the resulting conditional PDFs for our observation $Y$ are:
$$f_{Y|H_0}(y) = \frac{1}{\sqrt{2 \pi}} \exp\bigg(-\frac{(y+1)^2}{2}\bigg) \qquad \qquad f_{Y|H_1}(y) = \frac{1}{\sqrt{2 \pi}} \exp\bigg(-\frac{(y-1)^2}{2}\bigg)\ . $$ For this example, the ML rule has a very simple form
$$D_{\text{ML}}(y) =  \begin{cases} 1 & y \geq 0, \\ 0 & y < 0. \end{cases}$$ Let's start by spending a few minutes to understand how the ML rule behaves for different choices of the mean and variance. (You may need to install `sklearn` using the cell below.)

In [ ]:
#!pip install sklearn

In [ ]:
#numpy is a package for working with arrays, linear algebra, etc. and stands for "numerical Python".
import numpy as np
#pandas is a package for working with data frames.
import pandas as pd
#matplotlib is a plotting library that mimics Matlab conventions.
import matplotlib.pyplot as plt
#seaborn is a data visualization package.
import seaborn as sns
#scipy.stats is a statistics library
import scipy.stats as st
#sklearn is a machine learning library. (If you do not have it installed, uncomment the install cell above and run it.)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import zero_one_loss

The cell below is a custom function `gaussianMLrule` that takes in the means and variances for hypothesis 0 and hypothesis 1, and then plots the resulting conditional PDFs and the corresponding ML rule decision boundaries. You don't need to modify this cell, just run it to make it available.

In [ ]:
def gaussianMLrule(mean0,mean1,var0,var1,df=None,feature_col=None,label_col=None):
    #Create figure
    plt.gcf().set_size_inches(16, 6)
    
    #Plotting range 
    plot_ymin = np.min([mean0,mean1]) - 3*np.max([np.sqrt(var0),np.sqrt(var1)])
    plot_ymax = np.max([mean0,mean1]) + 3*np.max([np.sqrt(var0),np.sqrt(var1)])
    plot_yvalues = np.linspace(plot_ymin,plot_ymax,200)

    #Evaluate Gaussian PDF under hypothesis H0 and H1 over plotting range
    gaussian0 = st.norm.pdf(plot_yvalues,mean0,np.sqrt(var0))
    gaussian1 = st.norm.pdf(plot_yvalues,mean1,np.sqrt(var1))

    #Determine decision boundaries 
    decision_boundaries = np.roots([0.5*(1/var0 - 1/var1),mean1/var1 - mean0/var0,0.5*(mean0**2/var0 - mean1**2/var1 + np.log(var0/var1))])

    #Plot Gaussian PDFs and decisions boundaries
    plt.plot(plot_yvalues,gaussian0,color='blue',label='Label 0 Likelihood')
    plt.plot(plot_yvalues,gaussian1,color='red',label='Label 1 Likelihood')
    for boundary in decision_boundaries:
        plt.axvline(boundary,color='orange',linestyle='dashed',label='Decision Boundary')
    plt.xlabel('y')
    plt.ylabel('Likelihood')
    plt.legend()

    #If a dataframe is used as an input, then plot the 0/1 data as a scatter plot, mark the errors, and calculate the probability of error.
    if df is not None:
        df = df.copy()
        df_test0 = df[df[label_col]==0][feature_col] #extract data labeled 0
        df_test1 = df[df[label_col]==1][feature_col] #extract data labeled 1
        plt.scatter(df_test0, np.zeros_like(df_test0), color='blue',label='Test Data 0',s=100,facecolors='none')
        plt.scatter(df_test1, np.zeros_like(df_test1), color='red',label='Test Data 1',s=100,facecolors='none')
        df["guess"] = (st.norm.pdf(df[feature_col],mean0,np.sqrt(var0)) < st.norm.pdf(df[feature_col],mean1,np.sqrt(var1))).astype(int)
        errors_df = df[df[label_col] != df["guess"]][feature_col]
        plt.scatter(errors_df, np.zeros_like(errors_df), color='green',label='Errors', marker='*')
        plt.legend()
        numerrors = errors_df.shape[0]
        numtestvalues = df.shape[0]
        proberror = numerrors/numtestvalues
        print("Estimated Probability of Error = Fraction of Misclassified Points = " + str(numerrors) + "/" + str(numtestvalues) + " = " + str(proberror))

Now, let's plot the conditional PDFs and decision boundaries for our example above where $Y$ under $H_0$ is Gaussian$(-1,1)$ and $Y$ under $H_1$ is Gaussian$(+1,1)$.

In [ ]:
mean0 = +3
mean1 = +5
var0 = 1
var1 = 1
gaussianMLrule(mean0,mean1,var0,var1)

---
####  L8.1 Experiment with the means and variances above to see how the Gaussian ML rule behaves in different scenarios. Specifically, in your own words, answer the following questions:

#### (a) How many decision boundaries do you observe when both the means and variances are different?

#### Answer: 2

#### (b) How many decision boundaries do you observe when the means are different but the variances are equal?

#### Answer: 1

#### (c) How does the decision boundary depend on the means when the means are different and the variances are equal? Specifically, write a formula for the decision boundary that depends only on mean0 and mean1.

#### Answer: $$-\frac{(y-mean0)^2}{2} = -\frac{(y-mean1)^2}{2}$$ $$(y-mean0)^2 = (y-mean1)^2$$

#### (d) Using your answer from (c), predict where the decision boundary should be for mean0 = 3, mean1 = 5, var0 = var1 = 1. Does your guess agree with the plot?

#### Answer: 4
---

Now, let's generate some random data according to our models and see what happens. The cell below generates `n` data points per hypothesis and runs them through the ML rule. Any point that ends up crossing the decision boundary will be misclassified and result in an error. These points are marked with a green star, and we estimate the probability of error as the fraction of misclassified points. This estimate will change every time you run the cell, since new random data is generated and classified.

In [ ]:
n = 20 #number of data points generated per label (2*n points in total)
mean0 = -1
mean1 = +1
var0 = 3
var1 = 3
data_label0 = st.norm.rvs(mean0,np.sqrt(var0),n) #generate Gaussian(mean0,var0) data
data_label1 = st.norm.rvs(mean1,np.sqrt(var1),n) #generate Gaussian(mean1,var1) data
synthetic_data = {
    'y': np.append(data_label0,data_label1),
    'label': np.append(np.zeros_like(data_label0),np.ones_like(data_label1)).astype(int)
}
synthetic_df = pd.DataFrame(synthetic_data)
gaussianMLrule(mean0,mean1,var0,var1,synthetic_df,'y','label')

---
####  L8.2 Experiment with the means and variances above to see how the probability of error changes. Specifically, in your own words, answer the following questions:

#### (a) Try increasing the separation between mean0 and mean1 while keeping the variances fixed. What happens to the probability of error? 

#### Answer: The probability of the error decreases.

#### (b) Try increasing the variances while keeping the means fixed. (You can set var0 = var1 if you'd like.) What happens to the probability of error? 

#### Answer: The probability of the error increases.
---

In practice, we usually do not know how to set the means and variances in advance, even if we believe that the Gaussian distribution is a good model. The solution is quite intuitive: we can just fit the model parameters to the data. However, to ensure that we get a reasonable estimate of our real-world performance, it is common practice to partition our dataset in to a **training dataset** and a **testing dataset.** The training dataset can be used to select the probability model, fit its parameters, and make any other choices related to the decision rule. The testing dataset can **only** be used to evaluate the performance of the decision rule, and compare the performance of different decision rules to one another. Below is a simple example of a training and testing dataset.

In [ ]:
data = {
    'Y': [-1.3, -2.1, -2.7, 0.2, -1.1, -0.8, 2.3, 1.4, 0.6, 2.1, -0.2, 3.1],
    'label': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
}
df = pd.DataFrame(data)
train_df,test_df = train_test_split(df,test_size=0.5,stratify=df['label'],random_state=0)
print("Entire Data Frame")
display(df)
print("Training Data Frame (half of the rows from the original data frame, randomly selected)")
display(train_df)
print("Testing Data Frame (remaining half of the rows from the original data frame)")
display(test_df)

Now, we will use the training data to estimate the means and variances. The resulting conditional PDFs are plotted below along with the training data.

In [ ]:
train0_df = train_df[train_df['label']==0]['Y'] #extract data labeled 0
train1_df = train_df[train_df['label']==1]['Y'] #extract data labeled 1
#estimate means and variances
mean0 = train0_df.mean()
mean1 = train1_df.mean()
var0 = train0_df.var()
var1 = train1_df.var()
#plot conditional PDFs and ML decision boundary
gaussianMLrule(mean0,mean1,var0,var1)
#plot training data
plt.scatter(train0_df,np.zeros_like(train0_df), color='blue',label='Train Label 0',s=100)
plt.scatter(train1_df,np.zeros_like(train1_df), color='red',label='Train Label 1',s=100)

We can also plot the testing data along with the conditional PDFs and decision boundary. As before, green stars denote misclassified data points.

In [ ]:
gaussianMLrule(mean0,mean1,var0,var1,test_df,'Y','label')

What if believe that the variance should be the same in both models? We can combine our variance estimates together to make a **pooled variance** estimate as shown below. If there are $n_{\text{train},0}$ training data points labeled $0$ with variance estimate $\hat{\sigma_0}^2$ and $n_{\text{train},1}$ training data points labeled $1$ with variance estimate $\hat{\sigma_1}^2$, the pooled estimate is 
$$\hat{\sigma}^2_{\text{pooled}} = \frac{(n_{\text{train},0}-1) \hat{\sigma}_0^2 + (n_{\text{train},1}-1) \hat{\sigma}_1^2}{n_{\text{train},0} + n_{\text{train},1} - 2} \ . $$ This pooled variance is shown as an example below:

In [ ]:
ntrain0 = train0_df.shape[0]
ntrain1 = train1_df.shape[0]
var_pooled = ((ntrain0-1)*var0 + (ntrain1-1)*var1)/(ntrain0 + ntrain1 - 2)
gaussianMLrule(mean0,mean1,var_pooled,var_pooled)
#plot training data
plt.scatter(train0_df,np.zeros_like(train0_df), color='blue',label='Train Label 0',s=100)
plt.scatter(train1_df,np.zeros_like(train1_df), color='red',label='Train Label 1',s=100)

Let's again plot the testing data and see the probability of error.

In [ ]:
gaussianMLrule(mean0,mean1,var_pooled,var_pooled,test_df,'Y','label')

Let's go ahead and try out the Gaussian ML rule on the [breast cancer cell dataset] (https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic) (with both separate and pooled variance estimates). After loading the dataset, we drop the first three columns and change the diagnosis values from "B" to 0 and "M" to 1 to better align with our notation. 

In [ ]:
bc_df = pd.read_csv("datasets/breastcancercells.csv")
bc_df = bc_df.iloc[:,2:]
bc_df["Diagnosis"].replace("B",0,inplace=True)
bc_df["Diagnosis"].replace("M",1,inplace=True)
bc_df

Next, we partition the dataset into a training and testing dataset.

In [ ]:
bc_train_df, bc_test_df = train_test_split(bc_df,test_size=0.5,stratify=bc_df['Diagnosis'])

In [ ]:
display(bc_test_df)

In [ ]:
bc_train0_df = bc_train_df[bc_train_df['Diagnosis']==0]['Radius'] #extract data labeled 0
bc_train1_df = bc_train_df[bc_train_df['Diagnosis']==1]['Radius'] #extract data labeled 1
#estimate means and variances
mean0 = bc_train0_df.mean()
mean1 = bc_train1_df.mean()
var0 = bc_train0_df.var()
var1 = bc_train1_df.var()
#plot conditional PDFs and ML decision boundary
gaussianMLrule(mean0,mean1,var0,var1)
#plot training data
plt.scatter(bc_train0_df,np.zeros_like(bc_train0_df), color='blue',label='Train Label 0',s=100)
plt.scatter(bc_train1_df,np.zeros_like(bc_train1_df), color='red',label='Train Label 1',s=100)

In [ ]:
gaussianMLrule(mean0,mean1,var0,var1,bc_test_df,'Radius','Diagnosis')

In [ ]:
bc_ntrain0 = bc_train0_df.shape[0]
bc_ntrain1 = bc_train1_df.shape[0]
var_pooled = ((bc_ntrain0-1)*var0 + (bc_ntrain1-1)*var1)/(bc_ntrain0 + bc_ntrain1 - 2)
gaussianMLrule(mean0,mean1,var_pooled,var_pooled)
#plot training data
plt.scatter(bc_train0_df,np.zeros_like(bc_train0_df), color='blue',label='Train Label 0',s=100)
plt.scatter(bc_train1_df,np.zeros_like(bc_train1_df), color='red',label='Train Label 1',s=100)

In [ ]:
gaussianMLrule(mean0,mean1,var_pooled,var_pooled,bc_test_df,'Radius','Diagnosis')

---
####  L8.3 Using the code above as a template, evaluate the Gaussian ML rule on `bc_test_df`, specifically treating the "Radius" column as the observation Y.

#### (a) Generate a plot of the likelihoods, decision boundaries, test data, and errors using `gaussianMLrule` with separate variance estimates `var0` and `var1`. What is your estimate for the probability of error under the Gaussian ML rule when using separate variance parameters?

#### Answer: 0.154

#### (b) Generate a plot of the likelihoods, decision boundaries, test data, and errors using `gaussianMLrule` with a pooled variance estimate `var_pooled`. What is your estimate for the probability of error under the Gaussian ML rule when using a pooled variance parameter?

#### Answer: 0.147
---
(Hint: Your estimates will change every time you run the cell above, since the training and testing sets are randomly selected. You do not need to rerun the cell to get a particular answer, and it may be that QDA outperforms LDA on some runs but not others. This is due to both randomness in the training/testing split and the fact that the model is not a perfect fit to reality.)

Now, let's move from one to two features. As a running example, we'll stick with the breast cancer cell dataset. Above, we were using only the radius as a **feature** to make decisions. Let's now add the texture as a second feature. Hopefully, this will decrease the probability of error. The cell below is a custom function `gaussianMLrule2D` that incorporates all of the concepts we introduced above, including:
- Splitting the dataset into training and testing datasets.
- Estimating the mean vectors and covariance matrices.
- Applying the Gaussian ML rule for either pooled or separate covariance matrix estimates, based on the input `pooled_covariance`.
- Plotting the Gaussian contour plots, decision boundary, and the training data used to generate them.
- Plotting the decision boundary, testing data, and errors.
- Estimating the probability of error. 

You'll notice that the code and plots refer to **Linear Discriminant Analysis (LDA)** and **Quadratic Discriminant Analysis (QDA).** These are exactly the Gaussian ML rule with pooled and separate covariance matrix estimates, respectively. This terminology is more common in the machine learning literature, as well as in the built-in functions provided by `sklearn`.

In [ ]:
def gaussianMLrule2D(df,features_col1,features_col2,label_col,pooled_covariance=True, test_size=0.5):
    # Split data into training and testing sets
    features_df = df[[features_col1,features_col2]]
    label_df = df[label_col]
    features_train, features_test, label_train, label_test = train_test_split(features_df, label_df, test_size=test_size)
    
    features_train0 = features_train[label_train == 0]
    features_train1 = features_train[label_train == 1]
    mu0 = features_train0.mean()
    mu1 = features_train1.mean()
    Sigma0 = features_train0.cov()
    Sigma1 = features_train1.cov()

    # Fit LDA (pooled covariance matrix estimate) or QDA (separate covariance matrix estimates) classifier on training data
    if pooled_covariance:
        model = LinearDiscriminantAnalysis()
        n0 = features_train0.shape[0]
        n1 = features_train1.shape[0]
        Sigma_pooled = 1/(n0 + n1 - 2) * ((n0-1)*Sigma0 + (n1-1)*Sigma1)
        Sigma0 = Sigma_pooled
        Sigma1 = Sigma_pooled
    else:
        model = QuadraticDiscriminantAnalysis()

    model.fit(features_train, label_train)
    
    # Generate a meshgrid for contour plotting
    x_min, x_max = df[features_col1].min() - 1, df[features_col1].max() + 1
    y_min, y_max = df[features_col2].min() - 1, df[features_col2].max() + 1
    xpoints = np.linspace(x_min, x_max, 100)
    ypoints = np.linspace(y_min, y_max, 100)
    xx, yy = np.meshgrid(xpoints,ypoints)
    
    # Predict probabilities over the grid
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)
    
    # Start new figure for training data
    plt.figure(figsize=(10, 8))
    
    # Scatter plot of training 
    plt.scatter(features_train0[features_col1],features_train0[features_col2], c='blue', s=80, label='Train Label 0')
    plt.scatter(features_train1[features_col1],features_train1[features_col2], c='red', s=80, label='Train Label 1')

    # Contour plots of fitted jointly Gaussian PDFs
    pos = np.dstack((xx, yy))
    label0pdf = st.multivariate_normal(mu0,Sigma0).pdf(pos)
    plt.contour(xpoints,ypoints,label0pdf,cmap="Blues", linewidths=2, levels=6)
    label1pdf = st.multivariate_normal(mu1,Sigma1).pdf(pos)
    plt.contour(xpoints,ypoints,label1pdf,cmap="Reds", linewidths=2, levels=6)

    # Decision boundary
    plt.contour(xx, yy, Z, levels=[0.5], colors='black', linestyles='--')
    
    # Title, label axes, and legend
    plt.title(f"Gaussian Contours and {'LDA' if pooled_covariance else 'QDA'} Decision Boundary")
    plt.xlabel(features_col1)
    plt.ylabel(features_col2)
    plt.legend()

    # Start new figure for testing data
    plt.figure(figsize=(10, 8))
  
    # Decision boundary
    plt.contour(xx, yy, Z, levels=[0.5], colors='black', linestyles='--')

    # Scatter plot of testing
    features_test0 = features_test[label_test == 0]
    features_test1 = features_test[label_test == 1]
    plt.scatter(features_test0[features_col1], features_test0[features_col2], edgecolor='blue', facecolor='none', s=80, label='Test Label 0')
    plt.scatter(features_test1[features_col1], features_test1[features_col2], edgecolor='red', facecolor='none', s=80, label='Test Label 1')

    # Mark errors on the test set
    predictions = model.predict(features_test)
    errors = features_test[predictions != label_test]
    plt.scatter(errors[features_col1], errors[features_col2], color='green',marker ='*',s=80,label='Errors')
    
    # Title, label axes, and legend
    plt.title(f"{'LDA' if pooled_covariance else 'QDA'} Decision Boundary and Errors")
    plt.xlabel(features_col1)
    plt.ylabel(features_col2)
    plt.legend()
    
    # Display plot
    plt.show()
    
    # Calculate and output probability of error estimate
    error_df = features_test[predictions != label_test]
    numerrors = error_df.shape[0]
    numtestvalues = features_test.shape[0]
    proberror = numerrors/numtestvalues
    print("Estimated Probability of Error = Fraction of Misclassified Points = " + str(numerrors) + "/" + str(numtestvalues) + " = " + str(proberror))

Let's run this using the radius and texture columns as our two features. 

In [ ]:
gaussianMLrule2D(bc_df,"Radius","Texture","Diagnosis",pooled_covariance=True)

In [ ]:
gaussianMLrule2D(bc_df,"Radius","Texture","Diagnosis",pooled_covariance=False)

---
####  L8.4 For the scalar Gaussian ML rule, we observed in L8.1 that the decision rule takes a simpler form when the variances are equal. A similar phenomenon occurs here when the variance estimates are pooled (and thus equal). 

#### (a) Describe, in your own words, the form of the LDA decision boundary that results from using a pooled covariance matrix estimate.

#### Answer: The decision boundary is a straight line dividing between two data.

#### (b) Describe, in your own words, the form of the QDA decision boundary that results from using separate covariance matrix estimates. 

#### Answer: The decision boundary is a quadratic curve dividing between two data.
---

The code and questions above were carefully designed to help you develop visual intuition for how the Gaussian ML rule behaves. In general, there is no need to write code for well-known classifiers on our own, as sophisticated machine learning libraries like `sklearn` have already these available as built-in functions. For example, the code below simply outputs the probability of error estimates for LDA and QDA without any visualizations.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import zero_one_loss
# Split the dataset
features_train, features_test, label_train, label_test = train_test_split(bc_df[["Radius","Texture"]], bc_df["Diagnosis"], test_size=0.5)
# Initialize and train the LDA and QDA models
lda = LinearDiscriminantAnalysis()
lda.fit(features_train, label_train)
qda = QuadraticDiscriminantAnalysis()
qda.fit(features_train, label_train)
# Make predictions
lda_predictions = lda.predict(features_test)
qda_predictions = qda.predict(features_test)
#Calculate error probability
lda_proberror = zero_one_loss(lda_predictions,label_test)
qda_proberror = zero_one_loss(qda_predictions,label_test)
print("Estimated Probability of Error for LDA = " + str(lda_proberror))
print("Estimated Probability of Error for QDA = " + str(qda_proberror))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import zero_one_loss
# Split the dataset
features_train, features_test, label_train, label_test = train_test_split(bc_df.iloc[:,1:], bc_df["Diagnosis"], test_size=0.5)
# Initialize and train the LDA and QDA models
lda = LinearDiscriminantAnalysis()
lda.fit(features_train, label_train)
qda = QuadraticDiscriminantAnalysis()
qda.fit(features_train, label_train)
# Make predictions
lda_predictions = lda.predict(features_test)
qda_predictions = qda.predict(features_test)
#Calculate error probability
lda_proberror = zero_one_loss(lda_predictions,label_test)
qda_proberror = zero_one_loss(qda_predictions,label_test)
print("Estimated Probability of Error for LDA = " + str(lda_proberror))
print("Estimated Probability of Error for QDA = " + str(qda_proberror))

---
####  L8.5 Using the code above as a template, evaluate the LDA and QDA on `bc_df`, using all columns except for the "Diagnosis" column as features. Recall that you can select a subset of the dataset using row and column indices with `bc_df.iloc[ , ]`

#### (a) What is your estimate for the LDA probability of error?

#### Answer: 0.056

#### (b) What is your estimate for the QDA probabilty of error?

#### Answer: 0.063

#### (c) Overall, does the performance of the ML rule improve when it has access to additional features (i.e., observations)?

#### Answer: Yes because the probability of error for both LDA and QDA decreases.
---

---
####  L8.6 Finally, to make our grading process easier, please make sure your answers are written in the "Answer:" fields above and then clear the outputs of all cells before uploading your lab for grading. 
---
Instructions: 
In Jupyter Notebook or Jupyter Lab, use the "Edit" menu and then "Clear Outputs of All Cells".
In Google Colab, use the "Edit" menu and then "Clear all outputs".
